<style>
  body {
    color: black;
  }
  h1 {
    background-color: transparent;
    color: LightSteelBlue;
  }
  h3 {
    background-color: transparent;
    color: WhiteSmoke;
  }
  b {
    background-color: transparent;
    color: LightSteelBlue;
  }
</style>
<h1>Introduction</h1>
<h4>Column Names</h4>
<ul>
  <li><b>ArithmeticErrortarget:</b> the polarity of the tweet (0 = negative, 2 = neutral, 4 = positive)</li>
  <li><b>id:</b> The id of the tweet (2087)</li>
  <li><b>date:</b> The date of the tweet (Sat May 16 23:58:44 UTC 2009)</li>
  <li><b>flag:</b> The query (lyx). If there is no query, then this value is NO_QUERY.</li>
  <li><b>user:</b> The user that tweeted (robotickilldozr)</li>
  <li><b>text:</b> The text of the tweet (Lyx is cool)</li>
</ul>

<h4>Import</h4>

In [131]:
import csv
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

if __name__ == '__name__':
    print('Starting Spark Session. All data is loaded into memory')

<h4>Read the File</h4>

Read the csv file into a Python dataframe and then write it into a new file

In [132]:
df = pd.read_csv('.databases/tweets.csv', encoding='ISO-8859-1')
display(df)
display(df.shape)

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best feeling ever
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interviews! â« http://blip.fm/~8bmta
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me for details
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur


(1599999, 6)

<style>
  body {
    color: black;
  }
  h1 {
    background-color: transparent;
    color: LightSteelBlue;
  }
  h3 {
    background-color: transparent;
    color: WhiteSmoke;
  }
  b {
    background-color: transparent;
    color: LightSteelBlue;
  }
</style>
<h1>Action</h1>

Get the basic information about the content of the file

In [133]:
df.columns = ['sentiment', 'id', 'date', 'flag', 'user', 'text']
print('The first five rows of the database')
display(df.head())
print('Information')
display(df.info())
print('Description')
df.describe()

The first five rows of the database


,sentiment,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


Information
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599999 entries, 0 to 1599998
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1599999 non-null  int64 
 1   id         1599999 non-null  int64 
 2   date       1599999 non-null  object
 3   flag       1599999 non-null  object
 4   user       1599999 non-null  object
 5   text       1599999 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


None

Description


,sentiment,id
count,1.599999e+06,1.599999e+06
mean,2.000001e+00,1.998818e+09
std,2.000001e+00,1.935757e+08
min,0.000000e+00,1.467811e+09
25%,0.000000e+00,1.956916e+09
50%,4.000000e+00,2.002102e+09
75%,4.000000e+00,2.177059e+09
max,4.000000e+00,2.329206e+09


<h3>Point *</h3>Now let us get the number of unique values in each column; we had 

**df.unstack().groupby(level=0).nunique()**

as a suggestion but it was over 6 times slower

In [134]:
# we had 
# df.unstack().groupby(level=0).nunique()
# as a suggestion but it was over 6 times slower
df.apply(pd.Series.nunique)

sentiment          2
id           1598314
date          774362
flag               1
user          659775
text         1581465
dtype: int64

In [135]:
for _ in df.columns:
    display(_)
    display(df[_].value_counts())

'sentiment'

sentiment
4    800000
0    799999
Name: count, dtype: int64

'id'

id
2190457769    2
1974742852    2
2062516845    2
1551586713    2
1563681287    2
             ..
2197311343    1
2197311196    1
2197311146    1
2197310899    1
2193602129    1
Name: count, Length: 1598314, dtype: int64

'date'

...

flag
NO_QUERY    1599999
Name: count, dtype: int64

'user'

user
lost_dog           549
webwoke            345
tweetpet           310
SallytheShizzle    281
VioletsCRUK        279
                  ... 
iheartrobpattz       1
67trinity            1
Sibby                1
mAnyA_15             1
bpbabe               1
Name: count, Length: 659775, dtype: int64

'text'

text
isPlayer Has Died! Sorry                                                                              210
good morning                                                                                          118
headache                                                                                              115
Good morning                                                                                          112
Headache                                                                                              106
                                                                                                     ... 
braces  tell me it will be okay...                                                                      1
is stuck at home without curry                                                                          1
@mrsduryee I've applied to about 70 since I lost my job in March...it certainly FEELS like a lot!       1
The cheese I got @SarawithanR lost its sq


To see the whole width of the table we make a small permanent change

In [136]:
pd.set_option('display.max_colwidth', None)

Let us see if some of the functions work properly and how we can use them to gain more information about this database. Here we list all the tweets that are generated by the user 'lost_dog'. We choose to only print the texts and nothing else.

In [137]:
print(df[df['user'].astype('string') == 'lost_dog']['text'])

43934              @NyleW I am lost. Please help me find a good home. 
45573             @SallyD I am lost. Please help me find a good home. 
46918         @zuppaholic I am lost. Please help me find a good home. 
47948         @LOSTPETUSA I am lost. Please help me find a good home. 
50571     @JeanLevertHood I am lost. Please help me find a good home. 
                                      ...                             
792408       @trooppetrie I am lost. Please help me find a good home. 
793313         @Carly_FTS I am lost. Please help me find a good home. 
793609         @inathlone I am lost. Please help me find a good home. 
798607              @Kram I am lost. Please help me find a good home. 
799404         @W_Hancock I am lost. Please help me find a good home. 
Name: text, Length: 549, dtype: object


Find the 50 users that generated the most tweets

In [138]:
df_grouped = df.groupby(['user']).count()
df_sorted = df_grouped.sort_values(by=['user'], ascending=False)
df_largest = df_sorted.nlargest(50, 'flag')
display(df_largest)

,sentiment,id,date,flag,text
user,,,,,
lost_dog,549,549,549,549,549
webwoke,345,345,345,345,345
tweetpet,310,310,310,310,310
SallytheShizzle,281,281,281,281,281
VioletsCRUK,279,279,279,279,279
mcraddictal,276,276,276,276,276
tsarnick,248,248,248,248,248
what_bugs_u,246,246,246,246,246
Karen230683,238,238,238,238,238


<style>
  span {
    background-color: transparent;
    color: orange;
  }
</style>
What differs the tweets that share the same ID but are still saved in this dataframe, from the others, is that they have different sentiments; don't ask me why but some seem totally unreasonable. For instance, why should the following tweet, with the ID# 1467863684, be both positive and negative at the same time? Is it because they have mentioned the word "sad"?

<span>Awwh babs... you look so sad underneith that shop entrance of &quot;Yesterday's Musik&quot; O-: I like the look of the new transformer movie</span>

<h2>Investigating the Sentiments</h2>

As we see in Point * above, there are tweet IDs that are redundant. As a matter of fact, 1,598,314 out of 1,599,999 tweets have unique IDs. So, let us have a look at it and see what we can find out.

<h4>Step 1</h4>

Try and find how many tweets have more than one sentiment related to them. For this we count the occurrences of each value in the 'id' column

In [139]:
all_ids = df['id'].value_counts()
display(all_ids)

id
2190457769    2
1974742852    2
2062516845    2
1551586713    2
1563681287    2
             ..
2197311343    1
2197311196    1
2197311146    1
2197310899    1
2193602129    1
Name: count, Length: 1598314, dtype: int64

We then filter the result to include only values that appear twice or more

In [140]:
duplicated_tweet_ids = all_ids[all_ids >= 2]
display(duplicated_tweet_ids)

id
2190457769    2
1974742852    2
2062516845    2
1551586713    2
1563681287    2
             ..
2015412220    2
2006617256    2
1933201064    2
2189722020    2
1982279593    2
Name: count, Length: 1685, dtype: int64

This shows clearly that as we expected 1,599,999 - 1,598,314 = 1,685 tweets have double sentiments. So we print the rows in df for which the 'id' column is in repeated_values.index

In [141]:
duplicated_tweets = df[df['id'].isin(duplicated_tweet_ids.index)]
display(duplicated_tweets.sort_values(by='id').head(50))

,sentiment,id,date,flag,user,text
212,0,1467863684,Mon Apr 06 22:33:35 PDT 2009,NO_QUERY,DjGundam,Awwh babs... you look so sad underneith that shop entrance of &quot;Yesterday's Musik&quot; O-: I like the look of the new transformer movie
800260,4,1467863684,Mon Apr 06 22:33:35 PDT 2009,NO_QUERY,DjGundam,Awwh babs... you look so sad underneith that shop entrance of &quot;Yesterday's Musik&quot; O-: I like the look of the new transformer movie
274,0,1467880442,Mon Apr 06 22:38:04 PDT 2009,NO_QUERY,iCalvin,"Haven't tweeted nearly all day Posted my website tonight, hopefully that goes well Night time!"
800299,4,1467880442,Mon Apr 06 22:38:04 PDT 2009,NO_QUERY,iCalvin,"Haven't tweeted nearly all day Posted my website tonight, hopefully that goes well Night time!"
988,0,1468053611,Mon Apr 06 23:28:09 PDT 2009,NO_QUERY,mariejamora,@hellobebe I also send some updates in plurk but i upload photos on twitter! you didnt see any of my updates on plurk? Zero?
801279,4,1468053611,Mon Apr 06 23:28:09 PDT 2009,NO_QUERY,mariejamora,@hellobebe I also send some updates in plurk but i upload photos on twitter! you didnt see any of my updates on plurk? Zero?
1176,0,1468100580,Mon Apr 06 23:42:57 PDT 2009,NO_QUERY,cristygarza,good night swetdreamss to everyonee and jared never chat in kyte puff
801572,4,1468100580,Mon Apr 06 23:42:57 PDT 2009,NO_QUERY,cristygarza,good night swetdreamss to everyonee and jared never chat in kyte puff
1253,0,1468115720,Mon Apr 06 23:48:00 PDT 2009,NO_QUERY,WarholGirl,@ientje89 aw i'm fine too thanks! yeah i miss you so much on the MFC but hope we can talk later on today kisses :huglove:
801649,4,1468115720,Mon Apr 06 23:48:00 PDT 2009,NO_QUERY,WarholGirl,@ientje89 aw i'm fine too thanks! yeah i miss you so much on the MFC but hope we can talk later on today kisses :huglove:


Now we check if those rows with duplicate ids have one 0 and one 4 in the sentiment column. We do it by adding the sentiments for each duplicated tweet. If they all have 4 as the result of the aggregation, we can conclude that each duplicate has received one 0 and one 4 as sentiment value. Consequenlty

In [142]:
aggregated_sentiments = duplicated_tweets.groupby('id').agg({'sentiment': 'sum', 'text': 'first'})
display(aggregated_sentiments)

,sentiment,text
id,,
1467863684,4,Awwh babs... you look so sad underneith that shop entrance of &quot;Yesterday's Musik&quot; O-: I like the look of the new transformer movie
1467880442,4,"Haven't tweeted nearly all day Posted my website tonight, hopefully that goes well Night time!"
1468053611,4,@hellobebe I also send some updates in plurk but i upload photos on twitter! you didnt see any of my updates on plurk? Zero?
1468100580,4,good night swetdreamss to everyonee and jared never chat in kyte puff
1468115720,4,@ientje89 aw i'm fine too thanks! yeah i miss you so much on the MFC but hope we can talk later on today kisses :huglove:
...,...,...
2193278017,4,"oh dear HH is back please twitter do something about her. I'm begging you, please pretty please"
2193403830,4,"english exam went okay revising for french, r.e and geography now, urrff"
2193428118,4,"finally finished typing!!!! Woohoooo , still need to add graphs though"


Which can be tested by asking if the number of 4s in the 'sentiment' column is the same as the size of the table

In [143]:
aggregated_sentiments['sentiment'].value_counts()

sentiment
4    1685
Name: count, dtype: int64

And this means <h3>YES</h3> We can go on with the next step and try to remove one of the duplicates and change the sentiment of the other one to 2, which is the same as 'natural'

In [144]:
# remove those rows in df that have the same id as in duplicated_tweet_ids and a sentiment of 0
df_wihout_duplicates = df[~(df['id'].isin(duplicated_tweet_ids.index) & (df['sentiment'] == 0))]
display(df_wihout_duplicates)

,sentiment,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best feeling ever
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interviews! â« http://blip.fm/~8bmta
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me for details
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur


hidden text: <!--now we need to gather all those having the same ID and the same everything but the sentiment, which has received both 0 and 4. Some are clearly ambiguous; others not. So what we do is that <span>we just remove the redundancy and what is remained will receive a sentiment of 2</span>. A little bit risky, since it still may be a source of confusion for the learner.-->

<h1>TODO</h1>

Divide the 'date' into separate columns with the names 'day name', 'year', 'month', 'day', 'hour', 'minute', 'second'**

In [145]:
#Divide the 'date' from 'DayName MonthName Day %H:%M:%S TimeZone %Y' into separate columns with the names 'day name', 'year', 'month', 'day', 'hour', 'minute', 'second', 'time zone'
df['date'] = pd.to_datetime(df['date'], format='%a %b %d %H:%M:%S %z %Y')
df['day name'] = df['date'].dt.day_name()
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['hour'] = df['date'].dt.hour
df['minute'] = df['date'].dt.minute
df['second'] = df['date'].dt.second
df['time zone'] = df['date'].dt.tz
df['date'] = df['date'].dt.date

ValueError: time data "Mon Apr 06 22:19:49 PDT 2009" doesn't match format "%a %b %d %H:%M:%S %z %Y", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

Show the result, sorted by time zone in descending order

In [ ]:
df.sort_values(by='time zone', ascending=False).head(10)

,sentiment,id,date,flag,user,text,day name,year,month,day,hour,minute,second,time zone
0,0,1467810672,2009-04-06,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!,Monday,2009,4,6,0,0,0,None
1,0,1467810917,2009-04-06,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds,Monday,2009,4,6,0,0,0,None
2,0,1467811184,2009-04-06,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,Monday,2009,4,6,0,0,0,None
3,0,1467811193,2009-04-06,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.",Monday,2009,4,6,0,0,0,None
4,0,1467811372,2009-04-06,NO_QUERY,joy_wolf,@Kwesidei not the whole crew,Monday,2009,4,6,0,0,0,None
5,0,1467811592,2009-04-06,NO_QUERY,mybirch,Need a hug,Monday,2009,4,6,0,0,0,None
6,0,1467811594,2009-04-06,NO_QUERY,coZZ,"@LOLTrish hey long time no see! Yes.. Rains a bit ,only a bit LOL , I'm fine thanks , how's you ?",Monday,2009,4,6,0,0,0,None
7,0,1467811795,2009-04-06,NO_QUERY,2Hood4Hollywood,@Tatiana_K nope they didn't have it,Monday,2009,4,6,0,0,0,None
8,0,1467812025,2009-04-06,NO_QUERY,mimismo,@twittera que me muera ?,Monday,2009,4,6,0,0,0,None
9,0,1467812416,2009-04-06,NO_QUERY,erinx3leannexo,spring break in plain city... it's snowing,Monday,2009,4,6,0,0,0,None


**TODO : Find the ten dates when the most tweets were generated**

In [ ]:
doubleID_df = df.groupby('id').count() > 1
double_id = df.loc[df['id'].isin(doubleID_df.loc[doubleID_df['sentiment'] == True].index)]
double_id.sort_values(by='id').head(100)

**TODO : Find out the change in the frequnency of tweets per day**

**TODO : See if there is a correlation between the sentiment of the tweet and the day it was made**

**TODO : See if there is a correlation between the sentiment of the tweet and the time of the day it was made; eg if the tweets tens to be more negative at night compared to days and so on**

# TEST ZONE

In [ ]:
double_sentiment_id = df.groupby(['id']).count().sort_values(by='sentiment', ascending=False)
double_sentiment_id
#double_sentiment_id.loc[double_sentiment_id['sentiment'] > 1]
#df.groupby(['id']).count().sort_values(by='sentiment', ascending=False)

,sentiment,date,flag,user,text
id,,,,,
2013708350,2,2,2,2,2
1694607515,2,2,2,2,2
1981455916,2,2,2,2,2
1977625382,2,2,2,2,2
1557375700,2,2,2,2,2
...,...,...,...,...,...
1974380475,1,1,1,1,1
1974380466,1,1,1,1,1
1974380370,1,1,1,1,1


In [ ]:
doubleID_df = df.groupby('id').count() > 1
double_id = df.loc[df['id'].isin(doubleID_df.loc[doubleID_df['sentiment'] == True].index)]
double_id.sort_values(by='id').head(100)

,sentiment,id,date,flag,user,text
212,0,1467863684,Mon Apr 06 22:33:35 PDT 2009,NO_QUERY,DjGundam,Awwh babs... you look so sad underneith that shop entrance of &quot;Yesterday's Musik&quot; O-: I like the look of the new transformer movie
800260,4,1467863684,Mon Apr 06 22:33:35 PDT 2009,NO_QUERY,DjGundam,Awwh babs... you look so sad underneith that shop entrance of &quot;Yesterday's Musik&quot; O-: I like the look of the new transformer movie
274,0,1467880442,Mon Apr 06 22:38:04 PDT 2009,NO_QUERY,iCalvin,"Haven't tweeted nearly all day Posted my website tonight, hopefully that goes well Night time!"
800299,4,1467880442,Mon Apr 06 22:38:04 PDT 2009,NO_QUERY,iCalvin,"Haven't tweeted nearly all day Posted my website tonight, hopefully that goes well Night time!"
988,0,1468053611,Mon Apr 06 23:28:09 PDT 2009,NO_QUERY,mariejamora,@hellobebe I also send some updates in plurk but i upload photos on twitter! you didnt see any of my updates on plurk? Zero?
...,...,...,...,...,...,...
824538,4,1556267135,Sat Apr 18 22:12:27 PDT 2009,NO_QUERY,banerch,@meladiaz HAHAHAHAHA! ) i miss you toooooo! i have kwentooo! i'll tell you next time we see each other. :&gt;
17812,0,1556402800,Sat Apr 18 22:42:07 PDT 2009,NO_QUERY,StormClaudi,"@IzzyJ_Is_Here haha yayyy! aw, lady i really hope you're not sad. that's no fun just listen to calle ocho and dance dance dance!"
825096,4,1556402800,Sat Apr 18 22:42:07 PDT 2009,NO_QUERY,StormClaudi,"@IzzyJ_Is_Here haha yayyy! aw, lady i really hope you're not sad. that's no fun just listen to calle ocho and dance dance dance!"
825935,4,1556571114,Sat Apr 18 23:22:47 PDT 2009,NO_QUERY,Kikirowr,This little tree is tiiiiired 25's (and dealing with stupid people) tomorrow and then finishing 10s monday! Raiding every nice


In [ ]:
df.groupby('id').count()>1

,sentiment,date,flag,user,text
id,,,,,
1467810672,False,False,False,False,False
1467810917,False,False,False,False,False
1467811184,False,False,False,False,False
1467811193,False,False,False,False,False
1467811372,False,False,False,False,False
...,...,...,...,...,...
2329205009,False,False,False,False,False
2329205038,False,False,False,False,False
2329205473,False,False,False,False,False


In [ ]:
df.groupby('id')['id'].count()>1

id
1467810672    False
1467810917    False
1467811184    False
1467811193    False
1467811372    False
              ...  
2329205009    False
2329205038    False
2329205473    False
2329205574    False
2329205794    False
Name: id, Length: 1598314, dtype: bool

In [ ]:
pd.set_option('display.max_rows', 10000)

In [ ]:
df

,sentiment,id,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best feeling ever
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interviews! â« http://blip.fm/~8bmta
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me for details
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! Tupac Amaru Shakur


In [ ]:
# extract the row with the id number 1467811372
df.loc[df['id'] == 1467813782]

,sentiment,id,date,flag,user,text
18,0,1467813782,Mon Apr 06 22:20:34 PDT 2009,NO_QUERY,gi_gi_bee,@FakerPattyPattz Oh dear. Were you drinking out of the forgotten table drinks?


In [ ]:
# create a sample dataframe
daf = pd.DataFrame({'col1': [1, 2, 2, 3, 3, 3, 4, 4, 4, 4]})
display(daf)
# count the occurrences of each value in the 'col1' column
value_counts = daf['col1'].value_counts()
display(value_counts)
# filter the result to include only values that appear three times or more
repeated_values = value_counts[value_counts >= 3]
display(repeated_values)
# print the repeated values
print(repeated_values.index.tolist())

,col1
0,1
1,2
2,2
3,3
4,3
5,3
6,4
7,4
8,4
9,4


col1
4    4
3    3
2    2
1    1
Name: count, dtype: int64

col1
4    4
3    3
Name: count, dtype: int64

[4, 3]
